In [1]:
import pandas as pd

users = pd.read_sql(
    """SELECT * FROM public.martynov_features_lesson_22_users """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

posts = pd.read_sql(
    """SELECT * FROM public.martynov_post_features_lesson_22_posts """,
    con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
)

In [4]:
from catboost import CatBoostClassifier
from datetime import datetime
import pandas as pd

def prepare_features(user, posts, timestamp: datetime):
    user_features_series = user.iloc[0]
    posts_with_user_features = posts.assign(**user_features_series)
        
    # Преобразование timestamp в time_slot
    day_of_week = timestamp.weekday()
    hour = timestamp.hour
    time_slot = f"{day_of_week}_{hour}"

    # Добавляем time_slot к постам
    posts_with_user_features['time_slot'] = time_slot

    # Сохраняем post_id перед удалением
    post_ids = posts_with_user_features['post_id'].copy()

    # Удаляем ненужные столбцы, кроме post_id
    posts_with_user_features = posts_with_user_features.drop(['user_id', 'text', 'post_id'], axis=1)
    
    # Переупорядочиваем столбцы в соответствии с from_file.feature_names_, удаляем post_id для обучения
    features_for_prediction = posts_with_user_features.reindex(columns=from_file.feature_names_)
    
    return features_for_prediction, post_ids

# Подготавливаем данные
user = users[users['user_id'] == 200]
from_file = CatBoostClassifier()
from_file.load_model('catboost_model_2')
example_datetime = datetime(2021, 3, 14, 15, 30)


# Find the topic columns with values greater than 0
topic_columns = ['entertainment_target', 'movie_target', 'covid_target', 'sport_target',
                    'politics_target', 'business_target', 'tech_target']
user_topics = [col for col in topic_columns if user[col].values[0] > 0]

# Map the topic column names to the corresponding topic names in the 'posts' table
topic_mapping = {
    'entertainment_target': 'entertainment',
    'movie_target': 'movie',
    'covid_target': 'covid',
    'sport_target': 'sport',
    'politics_target': 'politics',
    'business_target': 'business',
    'tech_target': 'tech'
}
user_topic_names = [topic_mapping[topic] for topic in user_topics]

 # Filter the posts based on the user's preferred topics if they have more than 2 liked topics
if len(user_topic_names) > 1:
    filtered_posts = posts[posts['topic'].isin(user_topic_names)]
else:
    filtered_posts = posts
        
X, post_ids = prepare_features(user, filtered_posts, example_datetime)

# Получаем предсказания
probabilities = from_file.predict_proba(X)[:, 1]

# Создаем DataFrame с post_id и вероятностями
predictions_df = pd.DataFrame({'post_id': post_ids, 'probability': probabilities})
top5 = predictions_df.sort_values('probability', ascending=False).head(5)

# Объединяем top5 с posts по post_id
merged_top5 = pd.merge(top5, posts[['post_id', 'topic', 'text']], on='post_id')

# Возвращаем DataFrame


['movie', 'covid', 'sport', 'politics', 'tech']
